In [4]:
!pip -q install boto3


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
import boto3
import os
from collections import defaultdict

# -------------------------
# RunPod S3 Credentials
# -------------------------
ACCESS_KEY = "user_37sKcYrvnk9UXaIY3B3Zr90MH0g"
SECRET_KEY = "rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n"
BUCKET = "e9tcw5eupu"

ENDPOINT_URL = "https://s3api-eu-ro-1.runpod.io"
REGION = "eu-ro-1"  # required but not strictly enforced

# -------------------------
# S3 Client (IMPORTANT PART)
# -------------------------
s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT_URL,
    region_name=REGION,
)

In [6]:
def human_size(bytes_size):
    if bytes_size >= 1024**3:
        return f"{bytes_size / 1024**3:.2f} GB"
    elif bytes_size >= 1024**2:
        return f"{bytes_size / 1024**2:.2f} MB"
    elif bytes_size >= 1024:
        return f"{bytes_size / 1024:.2f} KB"
    return f"{bytes_size} B"


def list_tree_with_sizes(bucket):
    paginator = s3.get_paginator("list_objects_v2")

    tree = defaultdict(list)
    folder_sizes = defaultdict(int)
    total_size = 0

    for page in paginator.paginate(Bucket=bucket):
        for obj in page.get("Contents", []):
            key = obj["Key"]
            size = obj["Size"]

            top_folder = key.split("/")[0]
            tree[top_folder].append((key, size))

            folder_sizes[top_folder] += size
            total_size += size

    # PRINT TREE
    print(f"\n📦 Bucket: {bucket}\n")

    for folder, files in sorted(tree.items()):
        print(f"📂 {folder}/  ({human_size(folder_sizes[folder])})")
        for key, size in files:
            print(f"   └── {key}  [{human_size(size)}]")

    print("\n" + "=" * 50)
    print(f"TOTAL SIZE: {human_size(total_size)}")
    print("=" * 50)


list_tree_with_sizes(BUCKET)

ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: access denied: provided access key not allowed for bucket "e9tcw5eupu"